In [1]:
import sys
from sys import platform as _platform

if _platform == "win32":
    sys.path.insert(1, '..\prediction\markovchain\\')
else:
    sys.path.insert(1, '../prediction/markovchain/')


from MCpredict import MCpredict

import pandas as pd
import ipywidgets as widgets
from collections import defaultdict
import collections
from IPython.display import display
from IPython.display import clear_output
from ipywidgets import HBox, VBox

In [2]:
if _platform == "win32":
    predict_chain3 = MCpredict(3,5, '..\data\model_mc\\')
    predict_chain4 = MCpredict(4,5, '..\data\model_mc\\')
else:
    predict_chain3 = MCpredict(3,5, '../data/model_mc/')
    predict_chain4 = MCpredict(4,5, '../data/model_mc/')
input_sent_list = []

In [3]:
style = {'description_width': 'initial', 'select_width' : 'initial'}
variableType = None

In [4]:
def display_archive_types(archive_types_list):
    archive_types_list.sort()
    archive_types_list.insert(0,'Select')
    return archive_types_list

In [5]:
archive_type_dropdown = widgets.Dropdown(options = display_archive_types(list(predict_chain3.names_set[0])), description='Archive Type')

output_varType = widgets.Output()
output_proxy = widgets.Output()
output_units = widgets.Output()
output_int_var = widgets.Output()
output_int_var_det = widgets.Output()
output_inf_var = widgets.Output()
output_inf_var_units = widgets.Output()

var_type_dropdown = widgets.Dropdown(placeholder='Select',options = ['select','measured', 'inferred'], description='variable type'
        , style=style, ensure_option=True, disabled=False)
proxy_dropdown = widgets.Combobox(placeholder='Select',options = [], description='proxy observation type'
        , style=style, ensure_option=True, disabled=False)
units_dropdown = widgets.Combobox(placeholder='Select',options =[], description='units(Proxy Observation Type)'
        , style=style, ensure_option=True, disabled=False)
int_var_dropdown = widgets.Combobox(placeholder='Select',options = [], description='interpretation/variable'
        , style=style, ensure_option=True, disabled=False)
int_var_det_dropdown = widgets.Combobox(placeholder='Select',options = [], description='interpretation/variableDetail'
        , style=style, ensure_option=True, disabled=False)
inf_var_dropdown = widgets.Combobox(placeholder='Select',options = [], description='inferred variable'
        , style=style, ensure_option=True, disabled=False)
inf_var_units_dropdown = widgets.Combobox(placeholder='Select',options = [], description='units(Inferred Variable)'
        , style=style, ensure_option=True, disabled=False)

In [6]:
def inf_var_dropdown_eventhandler(change):
    global input_sent_list
    
    output_inf_var_units.clear_output()
    
    inf_var_type_val = change.new
    
    input_sent_list = input_sent_list[:4]
    input_sent_list.append(inf_var_type_val)
    input_sent = (',').join(input_sent_list)
    
    isInf = True if variableType == 'inferred' else False
    res = predict_chain4.predict_seq(input_sent, isInferred=isInf)['0']

    displayChildren(res, inf_var_units_dropdown, None, False, is_units=True)

    with output_inf_var_units:
        clear_output()
        display(inf_var_units_dropdown)
    

In [7]:
def int_var_det_dropdown_eventhandler(change):
    global input_sent_list

    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    int_var_det_value = change.new
    
    input_sent_list = input_sent_list[:3]
    input_sent_list.append(int_var_det_value)
    input_sent = (',').join(input_sent_list)
    
    isInf = True if variableType == 'inferred' else False
    res = predict_chain4.predict_seq(input_sent, isInferred=isInf)['0']

    displayChildren(res, inf_var_dropdown, inf_var_dropdown_eventhandler, True)

    with output_inf_var:
        clear_output()
        display(inf_var_dropdown)

In [8]:
def int_var_dropdown_eventhandler(change):
    global input_sent_list
    
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    int_var_type_value = change.new
    
    input_sent_list = input_sent_list[:2]
    input_sent_list.append(int_var_type_value)
    input_sent = (',').join(input_sent_list)
    
    isInf = True if variableType == 'inferred' else False
    res = predict_chain4.predict_seq(input_sent, isInferred=isInf)['0']

    displayChildren(res, int_var_det_dropdown, int_var_det_dropdown_eventhandler, True)

    with output_int_var_det:
        clear_output()
        display(int_var_det_dropdown)

In [9]:
def proxy_dropdown_eventhandler(change):
    global input_sent_list
    
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    proxy_obs_type_value = change.new
    
    input_sent_list = input_sent_list[:1]
    input_sent_list.append(proxy_obs_type_value)
    input_sent = (',').join(input_sent_list)
    
    isInf = True if variableType == 'inferred' else False
    res_chain3 = predict_chain3.predict_seq(input_sent, isInferred=isInf)['0']
    res_chain4 = predict_chain4.predict_seq(input_sent, isInferred=isInf)['0']
    
    displayChildren(res_chain3, units_dropdown, None, False, is_units=True)
    displayChildren(res_chain4, int_var_dropdown, int_var_dropdown_eventhandler, True)
    
    with output_units:
        clear_output()
        display(units_dropdown)
    with output_int_var:
        clear_output()
        display(int_var_dropdown)
    with output_int_var_det:
        clear_output()
    

In [10]:
def displayChildren(options_list, dropdown_obj, dropdown_eventhandler, to_observe, is_units = False):
    
    if is_units:
        if options_list and options_list[0] == 'NA':
            options_list = ['Unitless']
        else:
            options_list = [x if x != 'NA' else 'Unitless' for x in options_list]   
    else:
        if options_list and options_list[0] == 'NA':
            options_list = ['NA']

    dropdown_obj.options = options_list
    if to_observe:
        dropdown_obj.observe(dropdown_eventhandler, names='value')
    

In [11]:
def var_type_dropdown_eventhandler(change):
    global input_sent_list, variableType
    variableType = change.new
    
    output_proxy.clear_output()
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    isInf = True if variableType == 'inferred' else False

    if isInf:
        input_sent = (',').join(input_sent_list[:1])
        results = predict_chain4.predict_seq(input_sent, isInferred=isInf)['0']
    
        displayChildren(results, inf_var_dropdown, inf_var_dropdown_eventhandler, True)

        with output_inf_var:
            output_inf_var.clear_output()
            display(inf_var_dropdown)
            
    else:
        input_sent = (',').join(input_sent_list[:1])
        results = predict_chain3.predict_seq(input_sent, isInferred=isInf)['0']
    
        displayChildren(results, proxy_dropdown, proxy_dropdown_eventhandler, True)

        with output_proxy:
            output_proxy.clear_output()
            display(proxy_dropdown)

In [12]:
def archive_type_dropdown_eventhandler(change):
    global input_sent_list
    archive_type_value = change.new
    input_sent_list = []
    input_sent_list.append(archive_type_value)
    
    output_proxy.clear_output()
    output_units.clear_output()
    output_int_var.clear_output()
    output_int_var_det.clear_output()
    output_inf_var.clear_output()
    output_inf_var_units.clear_output()
    
    var_type_dropdown.observe(var_type_dropdown_eventhandler, names='value')
    with output_varType:
        output_varType.clear_output()
        display(var_type_dropdown)

In [13]:
archive_type_dropdown.observe(archive_type_dropdown_eventhandler, names='value')

display(archive_type_dropdown)

Dropdown(description='Archive Type', options=('Select', 'Coral', 'GlacierIce', 'Hybrid', 'LakeSediment', 'Mari…

In [14]:
zero_line = HBox([output_varType])
first_line = HBox([output_proxy])
second_line = HBox([output_units])
third_line = HBox([output_int_var, output_int_var_det])
fourth_line = HBox([output_inf_var, output_inf_var_units])
VBox([zero_line, first_line, second_line, third_line, fourth_line])